In [1]:
import os
import torch 
import numpy
import pandas as pd 
import cudf
import cupy
import nvtabular as nvt

### Create random input data similar to pre-processed Yoochoose dataset structure 

In [2]:
NUM_ROWS = 10000
session_length = 20
batch_size = 100
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(1, 51996, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    'purchase': numpy.random.randint(0, 2, NUM_ROWS)
    }
random_data = cudf.DataFrame(inputs)
random_data

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin,purchase
0,78416,6,5170,223,0.257485,0.515219,1
1,77392,5,33090,97,0.402454,0.347696,0
2,76310,4,45249,73,0.300941,0.485221,1
3,74044,6,22514,224,0.089426,0.971665,1
4,73174,4,20562,315,0.197405,0.038215,0
...,...,...,...,...,...,...,...
9995,76098,3,6114,74,0.275854,0.371962,1
9996,77689,7,14647,205,0.271153,0.140891,0
9997,79508,7,19443,37,0.660193,0.502686,0
9998,75131,4,44821,175,0.399550,0.021668,0


### NVTabular workflow

- #TODO : Change the workflow using tagging API once it is finalized  

In [3]:
# Define Groupby Workflow
groupby_features = list(inputs.keys()) >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list"],
        "category": ["list"],     
        "day": ["first"],
        "purchase": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")
# Trim sessions to first 20 items 
groupby_features_nonlist = [x for x in groupby_features.selector if '-list' not in x]
groupby_features_nonlist
groupby_features_trim = ((groupby_features - groupby_features_nonlist)) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

workflow = nvt.Workflow(groupby_features + groupby_features_trim )
dataset = nvt.Dataset(random_data, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

/nvtabular/nvtabular/workflow/node.py:43: FutureWarning: The `["a", "b", "c"] >> ops.Operator` syntax for creating a `ColumnGroup` has been deprecated in NVTabular 21.09 and will be removed in a future version.
  warnings.warn(


In [4]:
sessions_gdf.head(3)

,session_id,day-first,purchase-first,timestamp/weekday/sin-list,timestamp/age_days-list,category-list,item_id-list,timestamp/weekday/sin-list_trim,timestamp/age_days-list_trim,category-list_trim,item_id-list_trim
0,70000,7,1,[0.9977056992893304],[0.34988317423309656],[179],[7473],[0.9977056992893304],[0.34988317423309656],[179],[7473]
1,70001,6,0,"[0.17378161068345266, 0.9002091782718862]","[0.18362468961699863, 0.5071765117859279]","[214, 225]","[734, 10736]","[0.17378161068345266, 0.9002091782718862]","[0.18362468961699863, 0.5071765117859279]","[214, 225]","[734, 10736]"
2,70002,7,1,"[0.13081466554950227, 0.8583384532837279, 0.07...","[0.7766689797275483, 0.690512270122454, 0.7711...","[90, 273, 28]","[21937, 19305, 1242]","[0.13081466554950227, 0.8583384532837279, 0.07...","[0.7766689797275483, 0.690512270122454, 0.7711...","[90, 273, 28]","[21937, 19305, 1242]"


- We can save the workflow.

In [5]:
workflow.save('workflow_inference_test')

### Export pre-processed data by day 

In [6]:
# Convert to a Dataset and write out hive-partitioned data to disk
nvt_output_path_tmp ='./output_nvt_tmp/'
PARTITION_COL = 'day-first'
nvt.Dataset(sessions_gdf).to_parquet(nvt_output_path_tmp, partition_on=[PARTITION_COL])

In [7]:
OUTPUT_FOLDER = "./preproc_sessions_by_day_ts/"
!mkdir -p $OUTPUT_FOLDER
days_folders = [f for f in sorted(os.listdir(nvt_output_path_tmp)) if f.startswith(PARTITION_COL)]
for day_folder in days_folders:
    df = cudf.read_parquet(os.path.join(nvt_output_path_tmp, day_folder))
    out_folder = os.path.join(OUTPUT_FOLDER, day_folder.replace('day-first=', ''))
    os.makedirs(out_folder, exist_ok=True)
    df.to_parquet(os.path.join(out_folder, 'train.parquet'))
    
    random_values = cupy.random.rand(len(df))
    
    #Extracts 10% for valid and test set. Those sessions are also in the train set, but as evaluation
    #happens only for the subsequent day of training, that is not an issue, and we can keep the train set larger.
    valid_set = df[random_values <= 0.10]
    valid_set.to_parquet(os.path.join(out_folder, 'valid.parquet'))
    
    test_set = df[random_values >= 0.90]
    test_set.to_parquet(os.path.join(out_folder, 'test.parquet'))

# Transformers4rec model  

In [8]:
import torch 
import transformers4rec.torch as torch4rec
from transformers4rec.torch import TabularSequenceFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema
from transformers4rec.torch.head import NextItemPredictionTask
from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt

- Manually set the schema 

In [9]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_schema("schema.pb")

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [10]:
inputs = TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )

### End-to-end session-based Transformer-based model for item prediction:

- LM task + HF Transformer architecture + Next item-prediction task. 
- We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/head.py#L212) is the class to support next item prediction task.

In [11]:
#### Define XLNetConfig class and set default parameters 

# Set HF config of XLNet 
transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer=transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True),
    inputs=inputs,
    
)

# Get the end-to-end Model class 
model = torch4rec.Model(head, device='cuda')

# Non-incremental training and evaluation

- **Set Training arguments**

In [12]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True,
                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                  per_device_train_batch_size = 512, per_device_eval_batch_size = 32,
                  output_dir = ".", use_legacy_prediction_loop = False, max_sequence_length=20)
train_args.data_loader_engine = 'nvtabular'
train_args.shuffle = True

- **Define paths to train and eval data**

In [13]:
import glob
from transformers4rec.torch.utils.data_utils import PyarrowDataLoaderBuilder, NVTDataLoaderBuilder
train_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/train.parquet")
eval_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/valid.parquet")

- **Define Trainer**

In [14]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
    train_dataset_or_path=train_data_paths,
    eval_dataset_or_path=eval_data_paths,
)

- **Train the model**  

In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=15.00281016031901, metrics={'train_runtime': 2.1086, 'train_samples_per_second': 1.423, 'total_flos': 0.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 294912, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 498327552, 'train_mem_gpu_alloc_delta': 42733056, 'train_mem_cpu_peaked_delta': 45056, 'train_mem_gpu_peaked_delta': 424363520})

- **Compute evaluation metrics**

In [16]:
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

/opt/conda/lib/python3.8/site-packages/transformers/trainer_pt_utils.py:366: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


  epoch = 3.0
  eval_avgprecisionat_10 = 0.0
  eval_avgprecisionat_20 = 0.0
  eval_loss = 33.92699613069233
  eval_mem_cpu_alloc_delta = 2330624
  eval_mem_cpu_peaked_delta = 40960
  eval_mem_gpu_alloc_delta = -48640
  eval_mem_gpu_peaked_delta = 53957120
  eval_ndcgat_10 = 0.0
  eval_ndcgat_20 = 0.0
  eval_recallat_10 = 0.0
  eval_recallat_20 = 0.0
  eval_runtime = 2.3451
  eval_samples_per_second = 259.267


- **Compute Train metrics**

In [17]:
train_metrics = trainer.evaluate(eval_dataset=train_data_paths, metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


  epoch = 3.0
  eval_mem_cpu_alloc_delta = 1048576
  eval_mem_cpu_peaked_delta = 81920
  eval_mem_gpu_alloc_delta = 863232
  eval_mem_gpu_peaked_delta = 45685248
  train_avgprecisionat_10 = 0.0
  train_avgprecisionat_20 = 0.0
  train_loss = 34.69549894332886
  train_ndcgat_10 = 0.0
  train_ndcgat_20 = 0.0
  train_recallat_10 = 0.0
  train_recallat_20 = 0.0
  train_runtime = 2.7387
  train_samples_per_second = 233.685
